# 허깅페이스 트랜스포머

허깅페이스는 자연어를 통해 인공지능 기술의 민주화를 추구하는 조직입니다. 오픈소스 라이브러리인 "트랜스포머"는 자연어 처리 커뮤니티에서 큰 사랑을 받고 있고, 자연어 처리와 저연어 이해 작업에 아주 유용하고 강력합니다. 또한, 약 100여개의 언어에 대한 수천개의 사전 학습 된 모델을 제공합니다. 트랜스포머 라이브러리의 가장 강력한 장점은 PyTorch와 Tensorflow 모두와 호환이 되는 것입니다.

아래와 같이 pip 명령어를 사용하여 트랜스포머 라이브러를 바로 설치할 수 있습니다:

<font color='blue'>[역자]</font> 구글 Colab을 구글 드라이브와 연동하여 사용할 경우, 아래 명령어를 통해 구글 드라이브를 현재 Colab의 탐색기(? 컨테이너와 비슷)에 마운트할 수 있습니다.   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
#!pip install transformers==3.5.1
!pip install transformers # [역자]권장 명령어

이 책에서는 3.5.1 버전의 트랜스포머를 사용합니다. 이제 트랜스포머를 설치했으므로 설명을 시작하겠습니다. 

<font color='blue'>[역자]</font> 3.5.1 버전의 트랜스포머는 현재 Colab에서 기본적으로 지원하는 PyTorch의 버전인 1.10+cu11.0과 호환되지 않으므로, 번역일 기준(2022/04/05)으로 4.17.0 버전의 트랜스포머가 설치되었습니다. 트랜스포머 설치 시 버전을 명시하지 않을 경우, 안정적인 최신 버전의 트랜스포머가 설치되고 일반적으로 Colab에서 지원하는 최신 버전의 PyTorch와 호환이 되므로 트랜스포머 설치 시 버전을 명시하지 않는 것을 추천드립니다. 다만, 트랜스포머 라이브러리가 꾸준히 업데이트 되면서 일부 명령어가 다를 수 있습니다. 그것을 원하지 않으신 분은 트랜스포머 3.5.1버전과 호환되는 torch를 설치하시면 됩니다.

# BERT 임베딩 생성하기

이제 사전 학습된 BERT에서 임베딩을 추출하는 방법을 알아봅니다. "I love Paris (나는 파리를 사랑한다)"라는 문장이 있습니다. 허깅페이스의 트랜스포머 라이브러이와 함게 사전 학습된 BERT 모델을 사용해 문장에 있는 모든 단어의 문맥화된 단어 임베딩을 얻는 방법을 알아보겠습니다.

먼저, 필요한 모듈을 불러옵니다:

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

다음으로, 사전 학습된 BERT 모델을 다운로드 합니다. 사용할 수 있는 모든 사전 학습된 BERT 모델은 다음 링크에서 확인 할 수 있습니다 - https://huggingface.co/transformers/pre-trained_models.html. 우리는 'bert-base-uncased' 모델을 활용합니다. 모델 이름이 의미하는 바와 같이, 이 모델은 12개의 인코더로 구성되었고 대소문자 구별이 없는 토큰으로 학습을 하였습니다. BERT-base 모델을 활용하므로, 표현 사이즈는 768입니다.

사전 학습된 'bert-base-uncased' 모델을 다운로드하고 불러오세요:

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


다음으로, bert-based-uncased 모델을 사전 학습하는데 사용한 토크나이저를 다운로드하고 불러옵니다: 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

이제, BERT에 입력값을 넣기 전에 입력값을 어떻게 전처리하는지 알아보겠습니다.

## 입력값 전처리하기
아래 문장이 있습니다:

In [ ]:
sentence = 'I love Paris'

문장을 토큰화 하고 토큰들을 얻습니다

In [ ]:
tokens = tokenizer.tokenize(sentence)

토큰들을 출력해봅니다:

In [ ]:
print(tokens)

['i', 'love', 'paris']


이제, 토큰 리스트의 처음에 [CLS]과 끝에 [SEP]를 추가합니다:  

In [ ]:
tokens = ['[CLS]'] + tokens + ['[SEP]']

업데이트 된 토큰 리스트를 출력해봅니다:

In [ ]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]']


위에서 알 수 있듯이, [CLS] 토큰을 문두에 그리고 [SEP] 토큰을 추가하여 토큰들의 총 길이는 5가 됩니다.

토큰의 길이를 7로 유지하기 위해 [PAD] 토큰을 아래와 같이 리스트의 마지막에 추가합니다:


In [ ]:
tokens = tokens + ['[PAD]'] + ['[PAD]']

업데이트 된 토큰 리스를 출력해봅니다:

In [ ]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']


위에서 알 수 있듯이, [PAD] 토큰이 추가 된 토큰 리스트를 얻었고 그 길이는 7이 됩니다.

다음으로, 어텐션 마스크를 생성합니다. 토큰 리스트에서 해당 토큰이 [PAD]가 아니면 어텐션 마스크의 값을 1로 설정하고, 그렇지 않으면 아래에 보는 바와 같이 값을 0으로 설정합니다:



In [ ]:
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]

어텐션 마스크를 출력해봅니다:

In [ ]:
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


위에서 알 수 있듯이, [PAD] 토큰의 위치에는 어텐션 마스크의 값이 0이고 다른 경우엔 1이 됩니다.

다음으로, 아래에서 보듯이 모든 토큰을 해당하는 token_ids로 변환합니다:

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)

token_ids를 출렵해봅니다:

In [ ]:
print(token_ids)

[101, 1045, 2293, 3000, 102, 0, 0]


위 출력에서, 각 토큰은 유일한 토큰 아이디로 맵핑된 것을 확인할 수 있습니다.

이제, token_idx와 attention_mask를 아래에서 보는 바와 같이 텐서로 변환합니다:

<font color="blue">[역자]</font> torch.tensor.unsqueeze(0) 는 대상이 되는 텐서의 가장 낮은 dimension (0번째 dimension)을 하나 더 추가합니다. 즉, tensor.size()가 (7,)라면 unsqueeze()는 해당 텐서의 사이즈 (1,7)로 변환해줍니다. unsqueeze(0)를 하는 이유는, 입력 텐서는 기본적으로 0번째 dimension으로 batch_idx를 할당하기 때문입니다. tensor.size()를 통해 확인해보시길 바랍니다.

In [ ]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)
print(token_ids.shape)
print(attention_mask.shape)

torch.Size([1, 7])
torch.Size([1, 7])


다음으로, token_ids와 attention_mask를 사전 학습 된 BERT 모델에 넣어주고 임베딩을 얻습니다.

## 임베딩 얻기

아래 코드에서 보는 바와 같이, token_ids와 attention_mask를 모델에 넘겨주고 임베딩을 얻는다. 모델은 출력으로 두 개의 값을 튜플로 반환한다는 것을 주목하길 바랍니다. 첫번째 반환값(hidden_rep)은 은닉 상태의 표현을 나타내고 이는 마지막 인코더 (12번 인코더)로 부터 얻은 값으로 모든 입력 토큰에 대한 은닉 상태 표현입니다. 두번째 반환값(cls_head)은 [CLS] 토큰에 대한 은닉 상태 표현입니다. 

<font color="blue">[역자]</font> 트랜스포머 버전 이상에선, return_dict=False로 명시하여야 두 결과값을 튜플 타입으로 반환합니다. 명시하지 않을경우, return_dict=True로 다른 정보가 포함 된 딕셔너리 타입이 반환됩니다.

In [ ]:
#hidden_rep, cls_head = model(token_ids, attention_mask = attention_mask)
(hidden_rep,cls_head) = model(token_ids, attention_mask = attention_mask, return_dict=False)

위 코드에서, hidden_rep은 입력 문장의 모든 토큰에 대한 임베딩(은닉 상태 표현)을 포함합니다. hidden_rep 텐서의 사이즈를 출력해봅니다:

In [ ]:
print(hidden_rep.shape, cls_head.shape)

torch.Size([1, 7, 768]) torch.Size([1, 768])


텐서 사이즈 [1,7,768]의 각 요소는 [batch_size, sequence_length, hidden_size]를 나타냅니다.

입력 데이터의 배치 사이즈가 1이고, 문장에 7개의 토큰이 있으므로 문장의 길이(sequence_length)는 토큰의 개수와 같습니다. 그리고, hidden_size는 은닉 상태 표현(임베딩) 사이즈이고 BERT-base 모델에서 이 값은 768입니다.

은닉 상태 표현 벡터(hidden_rep)의 각 토큰의 의미는 아래와 같습니다:

- hidden_rep[0][0]는 입력 문장의 첫번째 토큰인 [CLS]에 대한 임베딩에 해당합니다.
- hidden_rep[0][1]은 입력 문장의 두번째 토큰인 'I'에 대한 임베딩에 해당합니다.
- hidden_rep[0][2]는 입력 문장의 세번째 토큰인 'love'에 대한 임베딩에 해당합니다.

이러한 방식으로 모든 토큰의 문맥적 표현을 얻을 수 있습니다. 이 문맥적 표현은 기본적으로 주어진 문장의 모든 단에 대한 문맥화 된 단어 임베딩을 의미합니다.

이제, cls_head에 대해 자세히 살펴보겠습니다. cls_head는 [CLS] 토큰의 표현을 함축합니다. cls_head의 사이즈를 출력해봅니다:

In [ ]:
print(cls_head.shape)

torch.Size([1, 768])


위 코드는 아래와 같이 출력 될 것 입니다:

toch.Size([1,768])

[1,768]은 각각 [batch_size, hidden_size]를 의미합니다.

cls_head는 문장의 종합적인 표현을 함축한다고 배웠습니다. 그래서, cls_head를 주어진 문장 'I love Paris'의 함축된 표현으로서 이를 대표하여 이용할 수 있습니다.

또한, 우리는 사전 학습된 BERT로부터 임베딩을 추출하는 방법을 익혔습니다. 그러나, 이러한 임베딩은 단지 BERT의 최상층 인코더 레이어(12번째 인코더)로부터 얻은 것입니다. BERT의 코든 인코더 레이어로부터 임베딩을 얻을 수도 있을까요? 답은 'Yes'입니다! 어떻게 할 수 있는지 다음 장에서 알아보도록 하겠습니다 (뾰로롱~).